In [16]:
import os
import pandas as pd
import pandas.api.types as ptypes
import numpy as np
import datetime as dt
flights_df_raw = pd.read_csv('assets/flights.csv', low_memory=False)
airports_df = pd.read_csv('assets/airports.csv')
airlines_df = pd.read_csv('assets/airlines.csv')



In [17]:
def data_preprocess(flights_df):

   
    df = flights_df.dropna(axis=0)
    airports = ['BOS', 'JFK', 'SFO', 'LAX']
    df = df[df["ORIGIN_AIRPORT"].isin(airports)]
    df = df[df['DEPARTURE_DELAY'] <= 1440]
    df["FLIGHT_NUMBER"] = df["FLIGHT_NUMBER"].astype(str)
    df['IS_DELAYED'] = df['DEPARTURE_DELAY'] >= 15
    
    def convert_to_datetime(row):
        time_str = str(int(row["SCHEDULED_DEPARTURE"])).zfill(4)
        hour = int(time_str[:2])
        minute = int(time_str[2:])
        return dt.datetime(row['YEAR'], row['MONTH'], row['DAY'], hour, minute)
    
    df['SCHEDULED_DEPARTURE'] = df.apply(convert_to_datetime, axis=1)
    df = df.drop(columns=['YEAR', 'MONTH', 'DAY'])
        
        
    

    

    return df
flights_df=data_preprocess(flights_df_raw)
print(flights_df)
print("Number of rows:", len(flights_df))

        ORIGIN_AIRPORT DESTINATION_AIRPORT AIRLINE FLIGHT_NUMBER  \
0                  LAX                 PBI      AA          2336   
1                  SFO                 CLT      US           840   
2                  LAX                 MIA      AA           258   
4                  SFO                 MSP      DL           806   
6                  LAX                 CLT      US          2013   
...                ...                 ...     ...           ...   
2966762            LAX                 ORD      AA           219   
2966766            LAX                 BOS      B6           688   
2966767            JFK                 PSE      B6           745   
2966768            JFK                 SJU      B6          1503   
2966770            JFK                 BQN      B6           839   

        SCHEDULED_DEPARTURE  DEPARTURE_DELAY  IS_DELAYED  
0       2015-01-01 00:10:00             -8.0       False  
1       2015-01-01 00:20:00             -2.0       False  
2     

In [13]:
def flights_per_airport(flights_df, airports_df):
 
    flights_df=data_preprocess(flights_df_raw)
    df= pd.merge(flights_df,airports_df, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE')
    df=df.groupby(['IATA_CODE']).size().reset_index(name="NUM_FLIGHTS")
    df = df.set_index('IATA_CODE')

    return df
flights_per_airport(flights_df, airports_df)


,NUM_FLIGHTS
IATA_CODE,
BOS,105276
JFK,91933
LAX,192584
SFO,145951


In [14]:
def top_three_airlines(flights_df, airlines_df):

    flights_df=data_preprocess(flights_df_raw)
   

    df = pd.merge(flights_df, airlines_df, left_on='AIRLINE', right_on='IATA_CODE', suffixes=('_flights', '_airlines'))
    
    df_grouped = df.groupby("IATA_CODE").apply(lambda x: pd.Series({
        'NUM_FLIGHTS': len(x),
        'PERC_DELAY': (x['DEPARTURE_DELAY'] >= 15).mean() * 100
    })).reset_index()
    
    df_merged = pd.merge(df_grouped, airlines_df[['IATA_CODE', 'AIRLINE']], on='IATA_CODE')
    df_merged.rename(columns={'AIRLINE':'AIRLINE_NAME'},inplace= True)
    sorted_airlines = df_merged.sort_values(by=['NUM_FLIGHTS', 'PERC_DELAY'], ascending=[False, True])
    
    top_three = pd.DataFrame(sorted_airlines[['AIRLINE_NAME', 'NUM_FLIGHTS', 'PERC_DELAY']].head(3).reset_index(drop=True))
    
    return top_three


delays = top_three_airlines(flights_df, airlines_df)
print(delays)

             AIRLINE_NAME  NUM_FLIGHTS  PERC_DELAY
0   United Air Lines Inc.      86562.0   23.265405
1         JetBlue Airways      85920.0   20.798417
2  American Airlines Inc.      77024.0   15.351059


In [15]:
def monthly_airport_delays(flights_df):
   
    flights_df = data_preprocess(flights_df_raw)
    flights_df['MONTH'] = flights_df['SCHEDULED_DEPARTURE'].dt.month_name()

    monthly_delays = flights_df.groupby(['MONTH', 'ORIGIN_AIRPORT']).agg(
        TOTAL_FLIGHTS=('IS_DELAYED', 'count'),
        DELAYED_FLIGHTS=('IS_DELAYED', 'sum')
    ).reset_index()

    monthly_delays['PERCENT_DELAY'] = (monthly_delays['DELAYED_FLIGHTS'] / monthly_delays['TOTAL_FLIGHTS']) 

    df = monthly_delays.pivot(index='MONTH', columns='ORIGIN_AIRPORT', values='PERCENT_DELAY')
    df.columns.name = None
    df.reset_index(inplace=True)

    months_order = ['January', 'February', 'March', 'April', 'May', 'June', 
                    'July', 'August', 'September', 'October', 'November', 'December']
    df['MONTH'] = pd.Categorical(df['MONTH'], categories=months_order, ordered=True)
    
    df.sort_values('MONTH', inplace=True)
    
    df.reset_index(drop=True,inplace=True)
    
    df['MONTH'] = df['MONTH'].astype(str)
    
    df.iloc[:, 1:] = df.iloc[:, 1:].round(4)



    return df
print(monthly_airport_delays(flights_df))

        MONTH     BOS     JFK     LAX     SFO
0     January  0.1902  0.2257  0.1738  0.2001
1    February  0.3248  0.3174  0.1978  0.2222
2       March  0.1984  0.2736  0.2246  0.1770
3       April  0.1553  0.2020  0.1855  0.1756
4         May  0.1552  0.1552  0.1990  0.2297
5        June  0.2032  0.1938  0.2474  0.2374
6        July  0.1968  0.2019  0.2772  0.2430
7      August  0.1988  0.1985  0.2289  0.2118
8   September  0.1274  0.1506  0.1486  0.1399
9    November  0.1177  0.1383  0.1741  0.1565
10   December  0.1982  0.2563  0.2627  0.2990
